In [2]:
import numpy as np
import sys
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris

In [3]:
# From ffnn.ipynb
def countSSError(results, expected_results):
    results = np.array(results)
    expected_results = np.array(expected_results)
    if results.shape != expected_results.shape:
        print("Error: array shape mismatch")
        sys.exit(1)
    else:
        return np.sum(np.square(results - expected_results))

Sklearn Using Iris Dataset

In [4]:
iris = load_iris()

learning_data = np.array(iris.data)
learning_target = np.array(iris.target)

classifier = MLPClassifier(
    hidden_layer_sizes=(5,3,3), 
    activation='relu', 
    max_iter=1000, 
    learning_rate='constant', 
    learning_rate_init=0.001, 
    random_state=1
    )

classifier.fit(learning_data, learning_target)
y_pred = classifier.predict(learning_data)

print("SKLearn MLPClassifier with 5,3,3,2 hidden layers and relu activation function")
print(f"Prediction:\n{y_pred}")
print()
print(f"Target:\n{learning_target}")
print()

weight_data = classifier.coefs_
bias_data = classifier.intercepts_

print("layer count: {}".format(classifier.n_layers_))
print("iteration count: {}".format(classifier.n_iter_))
print("output count: {}".format(classifier.n_outputs_))

print("Weight Input: \n", weight_data[0])
print("Bias Input: \n", bias_data[0])
print("Weight Output: \n", weight_data[2])
print("Bias Output: \n", bias_data[2])

error = countSSError(y_pred, learning_target)
print(f"Error: {error / len(learning_data) * 100} %")

SKLearn MLPClassifier with 5,3,3,2 hidden layers and relu activation function
Prediction:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]

Target:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]

Jumlah layer: 5
Jumlah iterasi: 1000
Jumlah fitur: 4
Jumlah output: 3
Weight Input: 
 [[-2.11237097e-02  3.13107969e-01 -1.55344599e-01 -4.62181600e-01
   2.39886078e-01]
 [-2.71512518e-01 -8.26239061e-01 -1.39626167e-04 -3.79599188e-01
   3.28595945e-01]
 [-2.10151629e-02  7.62541414e-0

c:\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Sklearn Using Test Cases

In [11]:
import json
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

In [12]:
def train_model(file_path):
    data = load_json(file_path)

    # Extract model parameters
    input_size = data["case"]["model"]["input_size"]
    layers = data["case"]["model"]["layers"]
    initial_weights = np.array(data["case"]["initial_weights"][0])  # Extracting weights from nested list
    target_output = np.array(data["case"]["target"])
    learning_rate = data["case"]["learning_parameters"]["learning_rate"]
    batch_size = data["case"]["learning_parameters"]["batch_size"]
    max_iteration = data["case"]["learning_parameters"]["max_iteration"]
    error_threshold = data["case"]["learning_parameters"]["error_threshold"]
    stopped_by = "max_iteration"

    # Initialize MLPRegressor model
    model = MLPRegressor(hidden_layer_sizes=[layer["number_of_neurons"] for layer in layers],
                         activation='identity',  
                         solver='adam',  
                         batch_size=batch_size,
                         learning_rate_init=learning_rate,
                         max_iter=max_iteration, 
                         random_state=1)

    print("Input Size:", input_size)
    print("Layers:", layers)
    print("Initial Weights:\n", initial_weights)
    print("Target Output:\n", target_output)
    print("Learning Rate:", learning_rate)
    print("Batch Size:", batch_size)
    print("Max Iteration:", max_iteration)
    print("Error Threshold:", error_threshold)
    print("\n------------------------------------\n")

    # Train the model
    model.fit(data["case"]["input"], target_output)
    predicted_output = model.predict(data["case"]["input"])
    error = mean_squared_error(target_output, predicted_output)

    # Check stopping criterion
    if error <= error_threshold:
        stopped_by = "error_threshold"

    print(data["case"]["input"])
    print(f"\n{target_output}")
    print(f"\n{predicted_output}")
    print("\n-------------------------------\n")

    # Get final weights
    final_weights = model.coefs_

    # Compare final weights with expected final weights
    expected_final_weights = np.array(data["expect"]["final_weights"][0])

    # Output stopping criterion and final weights
    print("Stopped by:", stopped_by)
    print("Final weights:")
    print(final_weights[1])
    print("Expected Weights")
    print(expected_final_weights)

In [13]:
file_path = "testcase/linear.json"
train_model(file_path)

Input Size: 2
Layers: [{'number_of_neurons': 3, 'activation_function': 'linear'}]
Initial Weights:
 [[ 0.1  0.3  0.2]
 [ 0.4  0.2 -0.7]
 [ 0.1 -0.8  0.5]]
Target Output:
 [[ 2.   0.3 -1.9]
 [ 1.3 -0.7  0.1]]
Learning Rate: 0.1
Batch Size: 2
Max Iteration: 1
Error Threshold: 0.0

------------------------------------

[[3.0, 1.0], [1.0, 2.0]]

[[ 2.   0.3 -1.9]
 [ 1.3 -0.7  0.1]]

[[-1.39816154 -0.42274121 -1.22749194]
 [ 0.02255002 -1.57869784  0.43272905]]

-------------------------------

Stopped by: max_iteration
Final weights:
[[-0.02236653 -0.06161119  0.47043898]
 [-0.69109547  0.65623505 -0.84522489]
 [ 0.24093502 -0.06539046  0.21737965]]
Expected Weights
[[ 0.22  0.36  0.11]
 [ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]


c:\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


In [143]:
import json
import tensorflow as tf

# Load the JSON data
with open('testcase/sigmoid.json') as f:
    data = json.load(f)

# Define the model architecture
class TensorModel(tf.keras.Model):
    def __init__(self, input_size, num_neurons, activation_function):
        super(TensorModel, self).__init__()
        self.dense = tf.keras.layers.Dense(num_neurons, activation=activation_function, use_bias=False)
        self(tf.keras.Input(shape=(input_size + 1,)))

    def call(self, inputs):
        return self.dense(inputs)

# Define the loss function
def loss_function(y_true, y_pred, activation_function):
    if activation_function == "softmax":
        loss_func = tf.keras.losses.CategoricalCrossentropy()
        return loss_func(y_true, y_pred)
    else:
        squared_error = tf.square(y_true - y_pred)
        loss = tf.reduce_sum(squared_error) / tf.cast(tf.shape(y_true)[0], tf.float32)
        return 0.5 * loss

# Parse JSON data
input_size = data["case"]["model"]["input_size"]
num_neurons = data["case"]["model"]["layers"][0]["number_of_neurons"]

# Extract input data from JSON and add bias
json_input = data["case"]["input"]
input_with_bias = [[1.0] + input_row for input_row in json_input]
input_data = tf.constant(input_with_bias, dtype=tf.float32)

# Extract target data from JSON and reshape it
json_target = data["case"]["target"]
target_data = tf.constant(json_target, dtype=tf.float32)
height, width = target_data.shape
target_data = tf.reshape(target_data, [1, height, width])

initial_weights = [tf.constant(weights, dtype=tf.float32) for weights in data["case"]["initial_weights"]]
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
activation_function = data["case"]["model"]["layers"][0]["activation_function"]

# Print the extracted data
# print("initial weights:", initial_weights)
# print("input data:", input_data)
# print("target data:", target_data)

# Initialize the model
model = TensorModel(input_size, num_neurons, activation_function)

# Set the initial weights for the dense layer
model.set_weights(initial_weights)
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

# Training
for iteration in range(max_iteration):
    total_loss = 0
    for i in range(0, input_data.shape[0], batch_size):
        x_batch = input_data[i:i+batch_size]
        y_batch = target_data[i:i+batch_size]

        with tf.GradientTape() as tape:
            predictions = model(x_batch, training=True)
            loss = loss_function(y_batch, predictions, activation_function)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        total_loss += loss
        
    avg_loss = total_loss / (input_data.shape[0] / batch_size) 

    # Check if the average loss is below the error threshold
    if avg_loss < error_threshold:
        print(f"Error threshold reached. Stopping training at iteration {iteration + 1}.")
        break

# Print the final weights and check
final_weights = model.dense.get_weights()[0]
expected_final_weights = data["expect"]["final_weights"]

print("Final Weights:")
print(final_weights)
print("Stopped by:", "max_iteration" if iteration == max_iteration - 1 else "error_threshold")
print("-------------------------------")


Final Weights:
[[0.23291178 0.06015345]
 [0.12884086 0.6484948 ]
 [0.837615   0.23158205]]
Stopped by: max_iteration
-------------------------------
